# Notebook for the LBFGS transfer learning on Duffing Equation

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
import json
from scipy.integrate import solve_ivp
from tqdm.auto import tqdm, trange
import os

# Add parent directory to sys.path
from pathlib import Path
current_path = Path.cwd()
parent_dir = current_path.parent.parent
sys.path.append(str(parent_dir))

# Import necessary modules
from src.utils_plot import plot_loss_and_all_solution, plot_transfer_learned_LBFGS

from src.load_save import load_run_history
from src.nonlinear_transfer_learning import GD_transfer_learning

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

In [ ]:
def check_versions_and_device():
  # set the device to the GPU if it is available, otherwise use the CPU
  current_dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  !nvidia-smi
  return current_dev

# set a global device variable to use in code
dev = check_versions_and_device()
print(dev)

### Load the pretrain model and history

In [ ]:
file_name = "nonlinear_repara_2141608"
equation_name = "duffing"

trained_model, \
x_range, \
iterations, \
hid_lay, \
num_equations, \
num_heads, \
loss_hist, \
alpha_list, \
A_list, \
IC_list, \
force_list = load_run_history(equation_name, file_name, dev, prev=False)

reparametrization=True
beta=0.5
def equation(t, y, alpha, beta=beta):
    if isinstance(y, torch.Tensor):
      yp = torch.zeros_like(y)
      force = torch.cos(t)
    elif isinstance(y, np.ndarray):
      yp = np.zeros_like(y)
      force = np.cos(t)
    yp[..., 0] = y[..., 1]
    yp[..., 1] = -0.1*y[..., 0] - alpha*y[..., 1] - beta*y[..., 0]**3 + force
    return yp
equation_list = [lambda t, y, Alpha=alpha: equation(t, y, Alpha) for alpha in alpha_list]

numerical_sol_fct = lambda x, v, alpha, beta: (solve_ivp(equation, [x_range[0], x_range[1]],
                                                    v.squeeze(), args=(alpha, beta), t_eval=x.squeeze(), method="Radau").y.T)
numerical_sol_list = [lambda x, IC=ic.detach().cpu().numpy(), Alpha=alpha, beta=beta: numerical_sol_fct(x, IC, Alpha, beta) for ic, alpha in zip(IC_list, alpha_list)]

plot_loss_and_all_solution(x_range=x_range, true_functs=numerical_sol_list,
                           trained_model=trained_model, IC_list=IC_list,
                           A_list=None, force=None, train_losses=loss_hist,
                           device=dev, equation_list=equation_list,
                           reparametrization=reparametrization)

## LBFGS transfer learning

### Choose transfer equation an learning parameters 

In [ ]:
alpha_transfer = 10
equation_transfer = lambda t, y, Alpha=alpha_transfer: equation(t, y, Alpha)
IC = IC_list[0]

lr=1
iterations=5
N=500

In [ ]:
loss_transfer, model_transfer, time_transfer = GD_transfer_learning(iterations=iterations, x_range=x_range, N=N,
                                                                    equation_transfer=equation_transfer, IC=IC,
                                                                    num_equations=num_equations, dev=dev, hid_lay=hid_lay,
                                                                    pretrained_model=trained_model, lr=lr, optimizer_name="LBFGS",
                                                                    decay=False, gamma=0.1, reparametrization=reparametrization, tqdm_bool=True)

### Plot learning result

In [ ]:
rng = np.random.default_rng()
t_eval = torch.arange(x_range[0], x_range[1], 0.001, requires_grad=True, device=dev).double()
t_eval = t_eval[np.concatenate(([0], rng.choice(range(1, len(t_eval)), size=512 - 1, replace=False)))]
t_eval = t_eval.reshape(-1, 1)
t_eval, _ = t_eval.sort(dim=0)

true_funct = lambda x: numerical_sol_fct(x, v=IC.detach().cpu().numpy(), alpha=alpha_transfer, beta=beta)
plot_transfer_learned_LBFGS(H=None, W_out=model_transfer, t_eval=t_eval, v=IC, A=None, force=None,
                            num_equations=num_equations, true_funct=true_funct,
                            transfer_loss=loss_transfer, reparametrization=reparametrization)

### LBFGS transfer leaning in several alpha regime

Choose training and equation parameters

In [ ]:
num_heads = 1
alpha_list = [5, 10, 15, 20, 25, 30, 35, 40]

lr_list = [1, 1, 1, 0.5, 0.2, 0.1, 0.1, 0.05]
iterations_list = [10, 15, 20, 20, 20, 20, 20, 30]
iterations_list = [3, 3, 3, 3, 3, 3, 3, 3]


### Transfer learning

In [ ]:
solution_PINNS = []
rng = np.random.default_rng()
t_eval = torch.arange(x_range[0], x_range[1], 0.001, requires_grad=True, device=dev).double()
t_eval = t_eval[np.concatenate(([0], rng.choice(range(1, len(t_eval)), size=512 - 1, replace=False)))]
t_eval = t_eval.reshape(-1, 1)
t_eval, _ = t_eval.sort(dim=0)

for i in trange(len(alpha_list)):
    equation_transfer = lambda t, y, Alpha=alpha_list[i]: equation(t, y, Alpha)
    _, model_transfer, _ = GD_transfer_learning(iterations=iterations_list[i], x_range=x_range, N=500,
                                             equation_transfer=equation_transfer, IC=IC_list[0],
                                             num_equations=num_equations, dev=dev, hid_lay=hid_lay,
                                             pretrained_model=trained_model, lr=lr_list[i], optimizer_name="LBFGS",
                                             decay=False, gamma=0.1, reparametrization=reparametrization, tqdm_bool=True)
    solution_PINNS.append(model_transfer(t_eval, reparametrization=reparametrization)[0])

### Plot MAE and MaxAE results over several alpha value

In [ ]:
mae_y1 = []
mae_y2 = []
maxae_y1 = []
maxae_y2 = []

for i in range(len(alpha_list)):
    true_funct = lambda x: numerical_sol_fct(x, v=IC_list[0].detach().cpu().numpy(), alpha=alpha_list[i], beta=beta)
    pinns = solution_PINNS[i].detach().cpu().numpy()
    numerical = true_funct(t_eval.detach().cpu().numpy())
    absolute_error = np.abs(pinns[:, 0, :] - numerical)
    mae_y1.append(absolute_error.mean(0)[0])
    mae_y2.append(absolute_error.mean(0)[1])
    maxae_y1.append(absolute_error.max(0)[0])
    maxae_y2.append(absolute_error.max(0)[1])

fig, ax = plt.subplots(1, figsize=(13, 4))

ax.plot(alpha_list, mae_y1, "-o", label="$MAE$ ${y_1}$", linewidth=2, markersize=6)
ax.plot(alpha_list, mae_y2,"-o", label="$MAE$ ${y_2}$", linewidth=2, markersize=6)
ax.plot(alpha_list, maxae_y1, "-x", color="#1f77b4", label="$MaxAE$ ${y_1}$", linewidth=2, markersize=8)
ax.plot(alpha_list, maxae_y2, "-x", color="#ff7f0e", label="$MaxAE$ ${y_2}$", linewidth=2, markersize=8)

#ax.set_xscale("log")
ax.set_yscale("log")
ax.set_title(r"Mean and Max Absolute Error with increasing Stiffness", fontsize=20)
ax.set_xlabel(r'Stiffness parameter $\alpha$', fontsize=16)
ax.set_ylabel('Absolute Error', fontsize=16)
ax.set_xticks(alpha_list)
ax.set_yticks([0.1, 0.01, 0.001, 0.0001],
              [r"$10^{-1}$", r"$10^{-2}$", r"$10^{-3}$", r"$10^{-4}$"])
ax.grid()
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=16)
ax.legend(loc='best', fontsize=14)

### Save MAE and MaxAE results over several alpha value

In [ ]:
history = {}
history["alpha_list"] = alpha_list
history["mae_y1"] = mae_y1
history["mae_y2"] = mae_y2
history["maxae_y1"] = maxae_y1
history["maxae_y2"] = maxae_y2

current_path = Path.cwd().parent.parent
path = os.path.join(current_path, "result_history")
with open(os.path.join(path, "Duffing_Error_Trained.json"),  "w") as fp:
    json.dump(history, fp)